## **Import Library**

In [1]:
!pip install google-play-scraper

# Mengimpor pustaka google_play_scraper untuk mengakses ulasan dan informasi aplikasi dari Google Play Store.
from google_play_scraper import app, reviews, Sort, reviews_all

# Pasang semua versi yang kompatibel:
!pip install sastrawi

import pandas as pd  # Pandas untuk manipulasi dan analisis data
pd.options.mode.chained_assignment = None  # Menonaktifkan peringatan chaining
import numpy as np  # NumPy untuk komputasi numerik
seed = 0
np.random.seed(seed)  # Mengatur seed untuk reproduktibilitas
import matplotlib.pyplot as plt  # Matplotlib untuk visualisasi data
import seaborn as sns  # Seaborn untuk visualisasi data statistik, mengatur gaya visualisasi
import xgboost as xgb
from wordcloud import WordCloud
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dropout, BatchNormalization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, GRU, Dense

import datetime as dt  # Manipulasi data waktu dan tanggal
import re  # Modul untuk bekerja dengan ekspresi reguler
import string  # Berisi konstanta string, seperti tanda baca
from nltk.tokenize import word_tokenize  # Tokenisasi teks
from nltk.corpus import stopwords  # Daftar kata-kata berhenti dalam teks
import requests
from io import StringIO

!pip install sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory  # Stemming (penghilangan imbuhan kata) dalam bahasa Indonesia
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory  # Menghapus kata-kata berhenti dalam bahasa Indonesia

from wordcloud import WordCloud  # Membuat visualisasi berbentuk awan kata (word cloud) dari teks

import nltk  # Import pustaka NLTK (Natural Language Toolkit).
nltk.download('punkt')  # Mengunduh dataset yang diperlukan untuk tokenisasi teks.
nltk.download('stopwords')  # Mengunduh dataset yang berisi daftar kata-kata berhenti (stopwords) dalam berbagai bahasa.
nltk.download('punkt_tab')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.6 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

## **Scraping Data**

In [2]:
# Mengimpor pustaka google_play_scraper untuk mengakses ulasan dan informasi aplikasi dari Google Play Store.
from google_play_scraper import app, reviews_all, Sort

# Mengambil semua ulasan dari aplikasi dengan ID 'com.mobile legend' di Google Play Store.
scrapreview = reviews_all(
    'com.lazada.android',    # ID aplikasi
    lang='id',               # Bahasa ulasan
    country='id',            # Negara
    sort=Sort.MOST_RELEVANT, # Urutan ulasan (default: Sort.MOST_RELEVANT)
    count=100000             # Jumlah ulasan yang ingin diambil
)

**Import file ke csv**

In [3]:
# Menyimpan ulasan dalam file CSV
import csv

with open('ulasan_aplikasi.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Review'])  # Menulis header kolom
    for review in scrapreview:
        writer.writerow([review['content']])  # Menulis konten ulasan ke dalam file CSV

## **Loading Dataset**

In [4]:
app_reviews_df = pd.DataFrame(scrapreview)
app_reviews_df.shape
app_reviews_df.head()
app_reviews_df.to_csv('ulasan_aplikasi.csv', index=False)

# Membuat DataFrame dari hasil scrapreview
app_reviews_df = pd.DataFrame(scrapreview)

# Menghitung jumlah baris dan kolom dalam DataFrame
jumlah_ulasan, jumlah_kolom = app_reviews_df.shape

In [5]:
# Menampilkan lima baris pertama dari DataFrame app_reviews_df
app_reviews_df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,5e546f3c-07b8-48a2-b6bd-06eb87328b8d,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Gimana mau belanja, update terus, tapi buruk k...",1,1,5.12,2018-09-23 20:08:52,Mohon maaf atas kendala teknis yang Anda alami...,2018-09-23 23:10:55,5.12
1,d4b43085-b00d-463e-8b75-4067c507acc0,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Sudah ke 4x nya dikecewakan sama program misi ...,1,23,7.73.0,2025-04-11 10:59:12,Kami sangat menghargai feedback Kakak tentang ...,2025-04-11 11:47:06,7.73.0
2,f824b66c-ea40-4472-beb5-3d82b1adf88a,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"kalo memang pengiriman sedang overload, fitur ...",1,8,7.72.0,2025-04-11 10:03:41,Terima kasih banyak atas masukan Kakak. Kami s...,2025-04-11 10:27:37,7.72.0
3,332a3a21-15c3-448d-9dc4-d727dcdb7d93,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,apk nya berat bgt jadi lemot. padahal hp pake ...,1,10,7.72.0,2025-04-10 04:29:03,Kami sangat menghargai masukan Kakak mengenai ...,2025-04-10 04:47:06,7.72.0
4,ad5e60cb-dd9e-4f14-9ecd-3ef0eff5fb54,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"dari awal sih gue ragu soal apk ini, pas gilir...",1,13,7.72.0,2025-04-10 12:23:01,Kami sangat menghargai masukan Kakak mengenai ...,2025-04-10 12:52:16,7.72.0


In [6]:
# Menampilkan informasi tentang DataFrame app_reviews_df
app_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112500 entries, 0 to 112499
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   reviewId              112500 non-null  object        
 1   userName              112500 non-null  object        
 2   userImage             112500 non-null  object        
 3   content               112500 non-null  object        
 4   score                 112500 non-null  int64         
 5   thumbsUpCount         112500 non-null  int64         
 6   reviewCreatedVersion  87693 non-null   object        
 7   at                    112500 non-null  datetime64[ns]
 8   replyContent          72581 non-null   object        
 9   repliedAt             72581 non-null   datetime64[ns]
 10  appVersion            87693 non-null   object        
dtypes: datetime64[ns](2), int64(2), object(7)
memory usage: 9.4+ MB


In [7]:
# Membuat DataFrame baru (cleans_df) dengan menghapus baris yang memiliki nilai yang hilang (NaN) dari app_reviews_df
cleans_df = app_reviews_df.dropna()

# Menghapus baris duplikat dari DataFrame cleans_df
cleans_df = cleans_df.drop_duplicates()

# Menghitung jumlah baris dan kolom dalam DataFrame clean_df setelah menghapus duplikat
jumlah_ulasan_setelah_hapus_duplikat, jumlah_kolom_setelah_hapus_duplikat = cleans_df.shape

In [8]:
cleans_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 57914 entries, 0 to 112499
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   reviewId              57914 non-null  object        
 1   userName              57914 non-null  object        
 2   userImage             57914 non-null  object        
 3   content               57914 non-null  object        
 4   score                 57914 non-null  int64         
 5   thumbsUpCount         57914 non-null  int64         
 6   reviewCreatedVersion  57914 non-null  object        
 7   at                    57914 non-null  datetime64[ns]
 8   replyContent          57914 non-null  object        
 9   repliedAt             57914 non-null  datetime64[ns]
 10  appVersion            57914 non-null  object        
dtypes: datetime64[ns](2), int64(2), object(7)
memory usage: 5.3+ MB


In [9]:
# Mengambil 12000 baris
clean_df = cleans_df.head(53471)
clean_df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,5e546f3c-07b8-48a2-b6bd-06eb87328b8d,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Gimana mau belanja, update terus, tapi buruk k...",1,1,5.12,2018-09-23 20:08:52,Mohon maaf atas kendala teknis yang Anda alami...,2018-09-23 23:10:55,5.12
1,d4b43085-b00d-463e-8b75-4067c507acc0,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Sudah ke 4x nya dikecewakan sama program misi ...,1,23,7.73.0,2025-04-11 10:59:12,Kami sangat menghargai feedback Kakak tentang ...,2025-04-11 11:47:06,7.73.0
2,f824b66c-ea40-4472-beb5-3d82b1adf88a,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"kalo memang pengiriman sedang overload, fitur ...",1,8,7.72.0,2025-04-11 10:03:41,Terima kasih banyak atas masukan Kakak. Kami s...,2025-04-11 10:27:37,7.72.0
3,332a3a21-15c3-448d-9dc4-d727dcdb7d93,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,apk nya berat bgt jadi lemot. padahal hp pake ...,1,10,7.72.0,2025-04-10 04:29:03,Kami sangat menghargai masukan Kakak mengenai ...,2025-04-10 04:47:06,7.72.0
4,ad5e60cb-dd9e-4f14-9ecd-3ef0eff5fb54,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"dari awal sih gue ragu soal apk ini, pas gilir...",1,13,7.72.0,2025-04-10 12:23:01,Kami sangat menghargai masukan Kakak mengenai ...,2025-04-10 12:52:16,7.72.0


In [10]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 53471 entries, 0 to 103446
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   reviewId              53471 non-null  object        
 1   userName              53471 non-null  object        
 2   userImage             53471 non-null  object        
 3   content               53471 non-null  object        
 4   score                 53471 non-null  int64         
 5   thumbsUpCount         53471 non-null  int64         
 6   reviewCreatedVersion  53471 non-null  object        
 7   at                    53471 non-null  datetime64[ns]
 8   replyContent          53471 non-null  object        
 9   repliedAt             53471 non-null  datetime64[ns]
 10  appVersion            53471 non-null  object        
dtypes: datetime64[ns](2), int64(2), object(7)
memory usage: 4.9+ MB


In [11]:
from google.colab import files
clean_df.to_csv('clean_df.csv', encoding = 'utf-8-sig')
files.download('clean_df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>